In [1]:
import torch.nn as nn 
import torch 
import torch.nn.functional as F
import torch.optim as optim 
import pandas as pd
import numpy as np 
import torchvision.transforms as transforms

In [2]:
class mnistdataset(torch.utils.data.Dataset):
    def __init__(self, file, transform=None):

        self.file = pd.read_csv(file)
        self.labels = self.file["label"].values
        self.transform = transform  
        

    def __len__(self):
        return self.file.shape[0]
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        im = self.file.iloc[idx, 1:].to_numpy(dtype="uint8").reshape(-1)
        im = np.array([im]).reshape(28,28)
        if self.transform:
            im = self.transform(im)
        return im, self.labels[idx]


In [3]:
train_transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(),transforms.Normalize((0.1307), (0.3081))])

In [4]:
train = mnistdataset("train.csv", transform = train_transform)

In [5]:
train_loader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.m1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.d1 = nn.Dropout2d(0.25)
        self.conv3 = nn.Conv2d(64,64,3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64,64,3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(64)
        self.m2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.d2 = nn.Dropout2d(0.25)
        self.conv6 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.d3 = nn.Dropout2d(0.25)
        self.lin1 = nn.Linear(4608, 400)
        self.d4 = nn.Dropout(0.4)
        self.lin2 = nn.Linear(400, 28)
        self.d5 = nn.Dropout(0.2)
        self.lin3 = nn.Linear(28, 10)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = self.m1(x)
        x = self.d1(x)
        x = F.relu(self.conv3(x))
        x = self.bn3(x)
        x = F.relu(self.conv4(x))
        x = self.bn4(x)
        x = F.relu(self.conv5(x))
        x = self.bn5(x)
        x = self.m2(x)
        x = self.d2(x)
        x = F.relu(self.conv6(x))
        x = self.bn6(x)
        x = self.d3(x)

        x = x.view(x.size(0), -1)
        
        x = F.relu(self.lin1(x))
        x = self.d1(x)
        x = F.relu(self.lin2(x))
        x = self.d2(x)
        x = self.lin3(x)

        return x



In [7]:
net = Net().cuda()
opt = optim.SGD(net.parameters(), lr= 0.01, momentum=0.5)
loss = nn.CrossEntropyLoss().cuda()

In [8]:
epochs = 10
for epoch in range(epochs):
    
    net.train()
    for batch_id, (im, target) in enumerate(train_loader):
    
        im = im.to('cuda')
        target = target.to('cuda')
        opt.zero_grad()
        pred = net(im)
        l = loss(pred, target)
        l.backward()
        opt.step()
        if (batch_id + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_id + 1) * len(im), len(train_loader.dataset),
                100. * (batch_id + 1) / len(train_loader), l.item()))
    

Train Epoch: 0 [1600/42000 (4%)]	Loss: 0.576881
Train Epoch: 0 [3200/42000 (8%)]	Loss: 0.170274
Train Epoch: 0 [4800/42000 (11%)]	Loss: 0.078572
Train Epoch: 0 [6400/42000 (15%)]	Loss: 0.062531
Train Epoch: 0 [8000/42000 (19%)]	Loss: 0.017799
Train Epoch: 0 [9600/42000 (23%)]	Loss: 0.118225
Train Epoch: 0 [11200/42000 (27%)]	Loss: 0.017775
Train Epoch: 0 [12800/42000 (30%)]	Loss: 0.017476
Train Epoch: 0 [14400/42000 (34%)]	Loss: 0.028269
Train Epoch: 0 [16000/42000 (38%)]	Loss: 0.021743
Train Epoch: 0 [17600/42000 (42%)]	Loss: 0.097567
Train Epoch: 0 [19200/42000 (46%)]	Loss: 0.245781
Train Epoch: 0 [20800/42000 (50%)]	Loss: 0.076020
Train Epoch: 0 [22400/42000 (53%)]	Loss: 0.015823
Train Epoch: 0 [24000/42000 (57%)]	Loss: 0.012150
Train Epoch: 0 [25600/42000 (61%)]	Loss: 0.028204
Train Epoch: 0 [27200/42000 (65%)]	Loss: 0.182551
Train Epoch: 0 [28800/42000 (69%)]	Loss: 0.006621
Train Epoch: 0 [30400/42000 (72%)]	Loss: 0.057527
Train Epoch: 0 [32000/42000 (76%)]	Loss: 0.002620
Train Ep

In [9]:
test = torch.tensor(pd.read_csv("test.csv").values)/255.0
test = test.reshape(28000, 28, 28).unsqueeze(1)


In [10]:
test = test.to('cuda')

In [11]:
a = []
for im in range(28000):
    
    out = net(test[None, im]).data.max(1, keepdim=True)[1]
    a.append(out)

In [12]:
a = torch.tensor(a)
a = a.numpy()

In [13]:
d = pd.DataFrame({"ImageId": range(1, 28001), "Label": a})

In [14]:
d.to_csv("submission.csv", index=False)